<a href="https://colab.research.google.com/github/Navam04/CSC490-Navam---Laurence/blob/main/CSC490_Navam_Laurence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Dependency Installation Commands Go Here:

!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.4 MB/s eta 0:00:00


In [2]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jangsienicajzkowy/afo-aerial-dataset-of-floating-objects")

print("Path to dataset files:", path)

Using Colab cache for faster access to the 'afo-aerial-dataset-of-floating-objects' dataset.
Path to dataset files: /kaggle/input/afo-aerial-dataset-of-floating-objects


In [3]:
import os
labelPath = path + '/PART_1/PART_1/6categories.names'
labels = {}
with open(labelPath, 'r') as f:
    l = f.read().splitlines()
    print(labels)
    for i in range(len(l)):
        labels[i] = l[i]

print(labels)

print(labelPath)

{}
{0: 'human', 1: 'wind/sup-board', 2: 'boat', 3: 'bouy', 4: 'sailboat', 5: 'kayak'}
/kaggle/input/afo-aerial-dataset-of-floating-objects/PART_1/PART_1/6categories.names


In [4]:
import os
from collections import Counter, defaultdict

# Path to the directory where images and labels are stored
labels_dir = path + '/PART_1/PART_1/6categories'
# Dictionary to hold counts: {image_filename: Counter({class_id: count, ...})}
image_counts = defaultdict(Counter)

# Loop over all annotation files
for fname in os.listdir(labels_dir):
    if fname.endswith('.txt'):
        img_name = fname.replace('.txt', '.jpg')
          # base name of image
        with open(os.path.join(labels_dir, fname), 'r') as f:
            for line in f:
                parts = line.strip().split()
                if not parts:
                    continue
                class_id = labels[int(parts[0])]
                image_counts[img_name][class_id] += 1

for img, counts in image_counts.items():
    print(f"{img}: {dict(counts)}")


k9_544.jpg: {'human': 1}
a_1152.jpg: {'human': 40, 'wind/sup-board': 6}
w2_227.jpg: {'boat': 1, 'human': 1, 'bouy': 1}
g_153.jpg: {'human': 3}
g_78.jpg: {'human': 2}
r2_7.jpg: {'kayak': 3}
r3_97.jpg: {'kayak': 1}
r3_88.jpg: {'kayak': 2}
r2_9.jpg: {'kayak': 3}
s1_256.jpg: {'human': 1}
r4_102.jpg: {'kayak': 3, 'human': 1}
w1_431.jpg: {'boat': 1, 'bouy': 1, 'wind/sup-board': 1, 'human': 1}
b1_226.jpg: {'human': 18, 'wind/sup-board': 1}
k1_39.jpg: {'human': 9}
s5_404.jpg: {'boat': 1}
z3_59.jpg: {'boat': 1, 'sailboat': 1}
r2_83.jpg: {'kayak': 3}
r2_36.jpg: {'kayak': 3}
c_121.jpg: {'wind/sup-board': 2, 'human': 18}
s2_37.jpg: {'human': 1}
r2_30.jpg: {'kayak': 3}
w2_215.jpg: {'human': 2}
i3_155.jpg: {'boat': 4}
a_1017.jpg: {'human': 41, 'wind/sup-board': 6}
s2_296.jpg: {'human': 1}
r3_245.jpg: {'kayak': 3}
z1_38.jpg: {'human': 4, 'boat': 1}
a_343.jpg: {'human': 42, 'wind/sup-board': 6}
r3_210.jpg: {'kayak': 2}
b1_45.jpg: {'wind/sup-board': 6, 'human': 31}
a_427.jpg: {'human': 1}
b1_169.jpg: {

In [5]:
#The total number of each class across dataset
total_counts = Counter()

for counts in image_counts.values():
    total_counts.update(counts)

print("Total object counts across dataset:")
for cls, cnt in total_counts.items():
    print(f"Class {cls}: {cnt}")


Total object counts across dataset:
Class human: 33174
Class wind/sup-board: 3922
Class boat: 702
Class bouy: 587
Class kayak: 1446
Class sailboat: 160


In [6]:
import os
import shutil

# ==== CONFIGURE THESE PATHS ====
labels_dir = path + "/PART_1/PART_1/6categories"       # Dir A: all labels (.txt)
splits_dir = path + "/PART_1/PART_1"       # Dir B: contains train.txt, val.txt, test.txt
images_dirs = [ path + "/PART_1/PART_1/images",
               path + "/PART_2/PART_2/images",
               path + "/PART_3/PART_3/images"]  # Dirs with images
output_base = "/content/dataset/sorted"  # Output root

# Ensure output directories exist
for split in ["train", "validation", "test"]:
    os.makedirs(os.path.join(output_base, split, "images"), exist_ok=True)
    os.makedirs(os.path.join(output_base, split, "labels"), exist_ok=True)

# Helper: find image in any of the image dirs
def find_image(base_name):
    exts = [".jpg", ".jpeg", ".png"]
    for d in images_dirs:
        for ext in exts:
            path = os.path.join(d, base_name + ext)
            if os.path.exists(path):
                return path
    return None

# Process each split
for split in ["train", "validation", "test"]:
    split_file = os.path.join(splits_dir, f"{split}.txt")
    with open(split_file, "r") as f:
        label_names = [line.strip() for line in f if line.strip()]

    for label_name in label_names:
        base, _ = os.path.splitext(label_name)  # e.g. "boat001"
        label_src = os.path.join(labels_dir, base + ".txt")
        label_dst = os.path.join(output_base, split, "labels", base + ".txt")

        # Copy label
        if os.path.exists(label_src):
            shutil.copy(label_src, label_dst)
        else:
            print(f"⚠️ Missing label {label_src}")

        # Copy matching image
        img_src = find_image(base)
        img_dst = os.path.join(output_base, split, "images", os.path.basename(img_src)) if img_src else None
        if img_src and os.path.exists(img_src):
            shutil.copy(img_src, img_dst)
        else:
            print(f"⚠️ No image found for {base}")


In [7]:
from ultralytics import YOLO

# 1. Load a small YOLO model (pretrained on COCO for transfer learning)
model = YOLO("yolov8s.pt")

# 2. Train on your dataset using data.yaml
results = model.train(
    data="data.yaml",
    epochs=5, #
    batch=8,     # Reduced batch size to 8 to prevent CUDA out of memory error
    imgsz=640,      # keep original resolution
    workers=2,    # parallel image loading works well on GPU
    patience=20
)


# 3. Validate the model (on the val split from data.yaml)
metrics = model.val()
print("\nValidation Results:")
print(metrics)  # prints precision, recall, mAP50, mAP50-95 etc.

# 4. Optionally run predictions on val set to see results visually
preds = model.predict(
    source="/content/dataset/sorted/validation/images",   # path to your validation images
    conf=0.25,                 # confidence threshold
    save=True                  # saves annotated images in runs/detect/predict
)

print("\nPredictions saved in:", preds[0].save_dir)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
Ultralytics 8.3.214 🚀 Python-3.12.12 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=8, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data.yaml, degrees=0.0, deterministic=True, device=None, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=5, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_